In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle


In [21]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


###### As we know that the first three rows as RowNumber, CustomerId and Surname are not required for the analysis or cannot play a role in prediction, we can remove them from the data frame.
#### Preprocess the data
Drop irrelevant features

In [22]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
## Here axis=1 means column delete
## If axis=2, it means row delete
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [23]:
# Encode the categorical variables (Gender, Geography)
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [24]:
## One hot encode 'Geography'
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [25]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [26]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography'])) 
## geo_encoder.toarray(), it means we are converting the sparse matrix to array for better prediction and to increase the efficiency.
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


#### Combining all one hot encoder columns with the orignal data

In [27]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


#### Save the encoders and scalers

In [ ]:
with open('label_encoder_gender.pkl', 'wb') as file:  ## here 'wb' means w->write b->binary
    pickle.dump(label_encoder_gender, file)  ## .dump() it means dump(convert) the object into the file in binary format

with open('onehot_encoder_geo.pkl', 'wb') as file:

    pickle.dump(onehot_encoder_geo, file)

In [ ]:
## Divide data into dependent and independent features
# x-> columns that are input for the prediction
# y-> columns that are output for the prediction
x=data.drop('Exited', axis=1)
y=data['Exited']

## Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# test_size=0.2, 20% data test ke liye, 80% training ke liye
# random_state=42, har baar same split mile (repeatable results)


## Scale these features (sab features ek jaisi scale mein hote hain (mean=0, std=1))
scaler= StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [35]:
x_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [36]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

##### Now, Data is ready and modified altogather

In [37]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


## ANN Implementation

In [40]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Activation
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard 
## EarlyStopping--> stop training automatically when the model stops improving. It's a way to prevent overfitting and save time

In [49]:
(x_train.shape[1],)
## As from the above data it shows the number of features in the dataset is 12. So, it tells the Hidden NN of layer 1 to get ready to take 12 inputs from the input layer.

(12,)

#### Building ANN model

In [45]:
model=Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)), ## First hidden layer HL1, connected with Input layer
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid') ## Output layer, connected with HL2
]
)

In [46]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### Compile the Model.
It tells about loss function, optimizer and Accuracy during the training.

In [52]:
import tensorflow as tf
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()
loss

In [53]:
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

#### Setup Tensorboard

In [ ]:
import datetime
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
## Tensorboard Setup
tensorflow_callback= TensorBoard(log_dir=log_dir, histogram_freq=1)

In [61]:
## Setup Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

#### Training the model

In [62]:
history=model.fit(
    x_train, y_train, validation_data=(x_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3393 - accuracy: 0.8590 - val_loss: 0.3395 - val_accuracy: 0.8605
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3380 - accuracy: 0.8602 - val_loss: 0.3467 - val_accuracy: 0.8605
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3350 - accuracy: 0.8619 - val_loss: 0.3357 - val_accuracy: 0.8655
Epoch 4/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3327 - accuracy: 0.8620 - val_loss: 0.3382 - val_accuracy: 0.8630
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3310 - accuracy: 0.8629 - val_loss: 0.3472 - val_accuracy: 0.8585
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3290 - accuracy: 0.8662 - val_loss: 0.3485 - val_accuracy: 0.8620
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3276 - accuracy: 0.8655 - val_loss: 0.3555 - val_accuracy: 0.8590

In [63]:
model.save('model.h5')

e:\GEN-AI Course\NLP\ANN Project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [65]:
## Load TensorBoard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [70]:
%tensorboard --logdir logs/fit20250805-213610

Reusing TensorBoard on port 6008 (pid 20948), started 0:00:06 ago. (Use '!kill 20948' to kill it.)

### Load pickle file
I had to load the pickle file and make the prediction

In [89]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [90]:
## Loading the trainned models
model=load_model('model.h5')

## Load the encoder and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:   ## Made earlier
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:    ## Made earlier
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file: ## Feature scaling info (Input values ko same range mein laane ke liye)
    scaler = pickle.load(file)

### Example Input Data

In [91]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [92]:
## Now the question arise is that how could I transform the geograpgy(geo) and gender in the above input_data as numbers(vectors) to have prediction. 
## Answer is that same as done before by onehot encodding and 

geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df 

e:\GEN-AI Course\NLP\ANN Project\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [93]:
## The input value is in key-value pairs now, so we have to convert it to the dataframe as

input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [94]:
## Encode categorical features for Gender now. As;

input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [95]:
## Combine one-hot encoded with input data

input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [96]:
## Scaling the input data (coverting it to array)

input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

#### Prediction(result)

In [101]:
prediction = model.predict(input_scaled) 
prediction

1/1 [==============================] - 0s 31ms/step


array([[0.03564366]], dtype=float32)

In [102]:
prediction_probab = prediction[0][0]
prediction_probab

0.035643656

In [103]:
if prediction_probab > 0.5:
    print("The customer is likely to churn")
else:
    print("The customer is unlikely to churn")

The customer is unlikely to churn
